In [ ]:
# This script scrapes the results of training and generates numbers in the
# format of Tables 1 and 2 in the paper (https://arxiv.org/abs/2103.13415).
# Numbers are slightly different because of implementation details and
# randomness across different runs of training.

In [ ]:
import os
import numpy as np
from google3.pyglib import gfile

In [ ]:
blender_scenes = ['chair', 'drums', 'ficus', 'hotdog', 'lego', 'materials', 'mic', 'ship']

def summarize_results(folder, scene_names, num_buckets):
  metric_names = ['psnrs', 'ssims', 'lpips']
  num_iters = 1000000
  precisions = [3, 4, 4, 4]

  results = []
  for scene_name in scene_names:
    test_preds_folder = os.path.join(folder, scene_name, 'test_preds')
    values = []
    for metric_name in metric_names:
      filename = os.path.join(folder, scene_name, 'test_preds', f'{metric_name}_{num_iters}.txt')
      with gfile.Open(filename) as f:
        v = np.array([float(s) for s in f.readline().split(' ')])
        values.append(np.mean(np.reshape(v, [-1, num_buckets]), 0))
    results.append(np.concatenate(values))
  avg_results = np.mean(np.array(results), 0)

  psnr, ssim, lpips = np.mean(np.reshape(avg_results, [-1, num_buckets]), 1)

  mse = np.exp(-0.1 * np.log(10.) * psnr)
  dssim = np.sqrt(1 - ssim)
  avg_avg = np.exp(np.mean(np.log(np.array([mse, dssim, lpips]))))

  s = []
  for i, v in enumerate(np.reshape(avg_results, [-1, num_buckets])):
    s.append(' '.join([f'{s:0.{precisions[i]}f}' for s in v]))
  s.append(f'{avg_avg:0.{precisions[-1]}f}')
  return ' | '.join(s)

In [ ]:
# The Multiscale Blender benchmark.
# These numbers roughly correspond to the "Mip-NeRF" row of Table 1.
print('mip-NeRF: ' + summarize_results('/cns/oz-d/home/barron/nerf/mipnerf/multiblender', blender_scenes, 4))

mip-NeRF: 32.634 34.350 35.505 35.636 | 0.9578 0.9703 0.9786 0.9834 | 0.0469 0.0260 0.0168 0.0120 | 0.0114


In [ ]:
# The (single-scale) Blender benchmark.
# These numbers roughly correspond to the "Mip-NeRF" row of Table 2.
print('mip-NeRF: ' + summarize_results('/cns/oz-d/home/barron/nerf/mipnerf/blender', blender_scenes, 1))

mip-NeRF: 33.085 | 0.9605 | 0.0425 | 0.0161
